
1. Датасет ml-latest
2. Вспомнить подходы, которые мы разбирали
3. Выбрать понравившийся подход к гибридным системам
4. Написать свою

# Загрузка данных

In [ ]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [ ]:
ratings.shape

(100836, 4)

In [ ]:
movies = pd.read_csv('movies.csv')
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [ ]:
movies.shape

(9742, 3)

In [ ]:
df = ratings.merge(movies)
print(df.shape)
df.head()

(100836, 6)


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


# Подготовка

In [ ]:
df = df.sort_values('timestamp').reset_index(drop=True)
df.head(3)

,userId,movieId,rating,timestamp,title,genres
0,429,595,5.0,828124615,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX
1,429,432,3.0,828124615,City Slickers II: The Legend of Curly's Gold (...,Adventure|Comedy|Western
2,429,227,3.0,828124615,Drop Zone (1994),Action|Thriller


In [ ]:
train = df.loc[:90000]
test = df.loc[90000:]

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366568 sha256=5708629a7dd063e5008fa0a499dab8e8179f4479e067e7cf2f57d290fcfe958f
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(df['rating'].min(), df['rating'].max()))
data_train = Dataset.load_from_df(train[['userId',	'movieId', 'rating']], reader)
data_test = Dataset.load_from_df(test[['userId',	'movieId', 'rating']], reader)

In [ ]:
data_train.df.shape, data_test.df.shape

((90001, 3), (10836, 3))

# SVD 

In [ ]:
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.model_selection import cross_validate

algo = SVD(n_factors=20, random_state=42)
trainset = data_train.build_full_trainset()
algo.fit(trainset)

In [ ]:
from surprise.model_selection import train_test_split

# sample random trainset and testset
trainset, testset = train_test_split(data_test, test_size=1.0)

In [ ]:
from surprise import accuracy

predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions, verbose=False)

1.0596496202199897

# Контент (genres)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()
train['genres'] = train['genres'].apply(lambda x: x.replace('|', ' '))
X_train = pd.DataFrame(tfidf.fit_transform(train['genres']).toarray(),
                       columns=tfidf.get_feature_names_out()).reset_index(drop=True)
y_train = train['rating']

<ipython-input-15-2a85e677b3bb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['genres'] = train['genres'].apply(lambda x: x.replace('|', ' '))


In [ ]:
import lightgbm as lgb

lgbm = lgb.LGBMRegressor()
lgbm.fit(X_train, y_train)

LGBMRegressor()

In [ ]:
test['genres'] = test['genres'].apply(lambda x: x.replace('|', ' '))
X_test = pd.DataFrame(tfidf.transform(test['genres']).toarray(),
                        columns=tfidf.get_feature_names_out()).reset_index(drop=True)
y_test = test['rating']

<ipython-input-17-ed85c5b0b91c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['genres'] = test['genres'].apply(lambda x: x.replace('|', ' '))


In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, lgbm.predict(X_test), squared=False)

1.0751577999871185

# Гибрид (blending)

In [ ]:
import numpy as np

def hybrid_rec(lgbm, X_test: pd.DataFrame, algo, testset: list, w1: float, w2: float)  -> np.array:
    x1 = lgbm.predict(X_test)

    x2 : list = []
    for i in algo.test(testset):
        x2.append(i.est)

    return w1 * np.array(x1) + w2 * np.array(x2)

In [ ]:
from sklearn.metrics import mean_squared_error

for i in range(1, 10):
    print(mean_squared_error(y_test, hybrid_rec(lgbm, X_test, algo, testset, w1=i*0.1, w2=1-i*0.1), squared=False))

1.1644633995023634
1.1469974931740832
1.131268569619437
1.1173499857091016
1.105310133498105
1.0952109744506826
1.0871065967029065
1.081041864742936
1.077051234042717
